In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
import sys
from deserialize_qapdata import *

$$\begin{aligned}
&\min_X f(X) = \textrm{tr}(A^\top XB X^\top)  \\
& = \textrm{tr}(X^\top A^\top XB) & x = \textrm{vec}(X)\\
& = \left <\textrm{vec}(X),  \textrm{vec}(A^\top X B )  \right > \\
& = \left <\textrm{vec}(X), B^\top \otimes A^\top \cdot \textrm{vec}(X)  \right > \\ 
& = x^\top (B^\top \otimes A^\top) x\\ 
& = \left<AX, XB\right> \\
\mathbf{s.t.} & \\ 
&X \in \Pi_{n}
\end{aligned}$$

# Conic optimization & geometric rounding

In [183]:
A0, B0 = parse('qapdata/bur26c.dat')
A = A0/np.linalg.norm(A0)
B = B0/np.linalg.norm(B0)
e = np.ones(shape=n)
E = np.ones(shape=(n, n))
ab = np.kron(B.T, A.T)
n, m = A.shape
S = np.block([[np.zeros((m, m)), 0.5 * np.eye(m)],
              [0.5 * np.eye(m), np.zeros((m, m))]]) + 1 * np.eye(2 * n)
K = np.linalg.cholesky(S)

In [203]:
def check_obj_val(x_sol):
    _obj = A0.T.dot(x_sol).dot(B0).dot(x_sol.T).trace()
    print(f'original obj {_obj}')
    return _obj

In [216]:
def co_gm(rd=True):    
    import mosek.fusion as mf
    expr = mf.Expr
    dom = mf.Domain
    mat = mf.Matrix

    model = mf.Model('qap')

    # X = model.variable([*A.shape], dom.inRange(0, 1))
    X = model.variable([*A.shape], dom.binary())
    Z = model.variable([*A.shape], dom.greaterThan(0.0))
    M = model.variable([*A.shape], dom.greaterThan(0.0))
    v = model.variable(1, dom.greaterThan(0.0))
    model.constraint(expr.sum(X, 0), dom.equalsTo(1))
    model.constraint(expr.sum(X, 1), dom.equalsTo(1))
    model.constraint(expr.sub(M, expr.mul(A, X)), dom.equalsTo(0))
    model.constraint(expr.sub(Z, expr.mul(X, B)), dom.equalsTo(0))

    x = expr.flatten(expr.mul(K, expr.vstack(Z, M)))
    model.constraint(expr.vstack(v, x), dom.inQCone())
    model.objective(mf.ObjectiveSense.Minimize, v)

    # set params
    model.setLogHandler(sys.stdout)
    model.setSolverParam("mioMaxTime", 60)
    model.acceptedSolutionStatus(mf.AccSolutionStatus.Anything)

    model.solve()

    model.flushSolutions()
    X_sol = X.level().reshape(A.shape)
    if rd:
        x, _ = extract_sol_rounding(X_sol, A, B)
        return x
    return X_sol

In [217]:
x_co_gm, _ = co_gm(False)
check_obj_val(x_co_gm)

Problem
  Name                   : qap             
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 2757            
  Cones                  : 1               
  Scalar variables       : 3383            
  Matrix variables       : 0               
  Integer variables      : 676             

Optimizer started.
Mixed integer optimizer started.
Threads used: 4
Presolve started.
Presolve terminated. Time = 0.97
Presolved problem: 2705 variables, 2080 constraints, 35360 non-zeros
Presolved problem: 0 general integer, 676 binary, 2029 continuous
Clique table size: 2028
BRANCHES RELAXS   ACT_NDS  DEPTH    BEST_INT_OBJ         BEST_RELAX_OBJ       REL_GAP(%)  TIME  
0        1        1        0        NA                   1.2049620320e+00     NA          1.2   
0        1        1        0        1.5312292963e+00     1.2049620320e+00     21.31       2.0   
0        1        1        0        1.5282876218e+00

SystemError: PyEval_EvalFrameEx returned a result with an error set

# $L_p$ regularization

various form of regularized problem:

- $\mathscr L_0$: $f(X) + \sigma ||X||_0$ is exact to the original problem for efficiently large $\sigma$ @jiang_l_p-norm_2016, but the problem itself is still NP-hard.
  
- $\mathscr L_p$: also suggested by @jiang_l_p-norm_2016, good in the sense:
  - strongly concave and the global optimizer must be at vertices
  - **local optimizer is a permutation matrix** if $\sigma, \epsilon$ satisfies some condition. Also, there is a lower bound for nonzero entries of the KKT points 

$$\min _{X \in D _{n}} F_{\sigma, p, \epsilon}(X):=f(X)+\sigma\|X+\epsilon 1 \|_{p}^{p}$$

- $\mathscr L_2$, and is based on the fact that $\Pi_n =  D_n  \bigcap \{X:\textrm{tr}(XX^\top) = n\}$, @xia_efficient_2010

$$\min_Xf(X)+\mu_{0} \cdot \textrm{tr} \left(X X^{\top}\right)$$


## $L_2$

$$\begin{aligned}
&tr(A^\top XB X^\top) + \mu_0 \cdot tr(X X^{\top}) \\
= & x^\top (B^\top \otimes A^\top + \mu I) x\\ 
&\textsf{this means RACQP is actually a LD method}
\end{aligned}$$
this means RACQP is actually a LD-like method. (but not exactly)

exact penalty, very likely to become a concave function, cannot be solved by conic solver
$$\begin{aligned}
&\textsf{exact penalty} \\
L_D = & f  + \mu_0\cdot | tr(XX^T) -  n| \\
& = f  + \mu_0\cdot n - \mu_0\cdot tr(XX^T)
\end{aligned}$$





In [211]:
def l2_subproblem(mu, rd=False):
    # do Cholesky
    P = np.kron(B.T, A.T) + mu * np.eye(n*n)
    U = np.linalg.cholesky(P)
    import mosek.fusion as mf
    expr = mf.Expr
    dom = mf.Domain
    mat = mf.Matrix

    model = mf.Model('qap')

    X = model.variable([*A.shape], dom.inRange(0, 1))
    m = expr.mul(U, expr.flatten(X))
    v = model.variable(1, dom.greaterThan(0.0))
    model.constraint(expr.sum(X, 0), dom.equalsTo(1))
    model.constraint(expr.sum(X, 1), dom.equalsTo(1))
    model.constraint(expr.vstack(v, m), dom.inQCone())
    model.objective(mf.ObjectiveSense.Minimize, v)

    # set params
    model.setLogHandler(sys.stdout)
    model.setSolverParam("mioMaxTime", 60)
    model.acceptedSolutionStatus(mf.AccSolutionStatus.Anything)

    model.solve()

    model.flushSolutions()
    X_sol = X.level().reshape(A.shape)
    if rd:
        x, _ = extract_sol_rounding(X_sol, A, B)
        return x
    return X_sol

In [221]:
X_sol = l2_subproblem(mu, True)


check_obj_val(X_sol)

Problem
  Name                   : qap             
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 729             
  Cones                  : 1               
  Scalar variables       : 1355            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.01            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.05    
Problem
  Name                   : qap             
  Objective sense        : min             
  Type                   : CONIC (conic optimizat

5657757.0

In [222]:
X_sol.dot(X_sol.T).trace()

26

$$\begin{aligned}
&  \nabla f = A^\top XB + AXB^\top \\
& \nabla tr(XX^\top) = 2X
\end{aligned}$$